<a href="https://colab.research.google.com/github/rai-sandeep/ai-colab-notebooks/blob/main/Training-exp2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install

In [1]:
# https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

model_id = "vilsonrodrigues/falcon-7b-instruct-sharded"
trained_model_id = "falcon-7b-instruct-sharded-trained-v1"
ds_id = "rai-sandeep/dataset_full_v2"
example_ques = "Generate a White Paper ABSTRACT on Modernizing Enterprise Systems."
example_ques2 = "Generate a White Paper MAINCONTENT on Bitcoin Mining."
max_tok = 200

save_trained_model=False
%env HF_TOKEN=<ENTER-HUGGINGFACE-TOKEN-HERE>

env: HF_TOKEN=<ENTER-HUGGINGFACE-TOKEN-HERE>


In [2]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.1/97.1 MB 10.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.5/268.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 10.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Import

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import transformers
import torch
from torch.utils.data import DataLoader, Dataset
import torch
from transformers import AutoTokenizer


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('8013'), PosixPath('http')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//colab.research.google.com/tun/m/cc483011

# Load Quantized Model

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0}, trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [5]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [7]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 4718592 || all params: 3613463424 || trainable%: 0.13058363808693696


# Prepare Data

In [8]:
from datasets import load_dataset

data = load_dataset(ds_id)

ds_len=len(data['train'])
rows_sel=data['train'].select(range(1))
rows_sel['format']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/26 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/rai-sandeep___parquet/rai-sandeep--dataset_full_v2-168b8fcb3c3cf62c/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

['As digital technologies, smart wearables and remote monitoring capabilities penetrate healthcare, traditional healthcare companies are unable to keep up with end-user expectations. Under pressure to adopt rapid transformation, these organizations are looking for robust and end-to-end testing procedures. This paper explains various end-to-end testing approaches within the four main modernization techniques for healthcare companies. The analysis presented here acts as a guideline for healthcare leaders to make strategic and informed decisions on how to modernize their systems based on the needs of their end-users.']

In [9]:
tokenizer.pad_token = tokenizer.eos_token

# prompt_template = "### Instruction: {prompt}\n### Response:"

train_dataset = data['train'].select(range(ds_len)).map(lambda x: {"input_text": x['task']  + "\n" + x['format']})

# Tokenize the datasets
train_encodings = tokenizer(train_dataset['input_text'], max_length=256, truncation=True, padding=True, return_tensors='pt')

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [10]:
  class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [11]:
# Convert the encodings to PyTorch datasets
train_dataset = TextDataset(train_encodings)

# Example Before Fine Tuning

In [12]:
def generate(index):

  print("Question:")
  print(example_ques)

  encoding = tokenizer(example_ques, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print("Question2:")
  print(example_ques2)

  encoding = tokenizer(example_ques2, return_tensors="pt").to("cuda:0")
  output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

  print("Answer:")
  print(tokenizer.decode(output[0], skip_special_tokens=True))

  print()

In [13]:
generate(0)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1264: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Generate a White Paper ABSTRACT on Modernizing Enterprise Systems.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer:
Generate a White Paper ABSTRACT on Modernizing Enterprise Systems.
Modernizing enterprise systems is a critical challenge for businesses today. As organizations strive to keep up with the latest technology trends, they need to ensure their systems are efficient, scalable, and adaptable to changing market conditions. This white paper explores the challenges and opportunities associated with modernizing enterprise systems, providing insights on how businesses can achieve a competitive advantage through this process.
Question2:
Generate a White Paper MAINCONTENT on Bitcoin Mining.
Answer:
Generate a White Paper MAINCONTENT on Bitcoin Mining.
Bitcoin mining is the process by which transactions are verified and added to the public ledger. It involves solving complex mathematical equations in order to validate the transactions and receive a reward in the form of newly issued bitcoins. This white paper will provide an in-depth analysis of the process, including the hardware and softwa

# Training

In [14]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    # eval_dataset=val_dataset,
    args=transformers.TrainingArguments(
        num_train_epochs=50,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=4,
        warmup_ratio=0.05,
        # max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        lr_scheduler_type='cosine',
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

#import gc
#del train_encodings, data
#gc.collect()
#torch.cuda.empty_cache()

trainer.train()

<ipython-input-10-bd500a371ea7>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
1,1.588300
2,1.584400
3,1.579500
4,1.566500
5,1.542400
6,1.528600
7,1.526000
8,1.517600
9,1.513000
10,1.504200


TrainOutput(global_step=50, training_loss=1.2671170830726624, metrics={'train_runtime': 973.2738, 'train_samples_per_second': 1.336, 'train_steps_per_second': 0.051, 'total_flos': 6625371154022400.0, 'train_loss': 1.2671170830726624, 'epoch': 50.0})

# Example After Fine Tuning

In [15]:
model.config.use_cache = True
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): RWForCausalLM(
      (transformer): RWModel(
        (word_embeddings): Embedding(65024, 4544)
        (h): ModuleList(
          (0-31): 32 x DecoderLayer(
            (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
            (self_attention): Attention(
              (maybe_rotary): RotaryEmbedding()
              (query_key_value): Linear4bit(
                in_features=4544, out_features=4672, bias=False
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4544, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4672, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): Parameter

In [16]:
#example_ques = "Generate a White Paper abstract on Smarter Way To Build System Resilience."
#example_ques2 = "Generate a White Paper abstract on Smarter Way To Build System Resilience. Include the following keywords: available, futuristic, evolving."

generate(0)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Question:
Generate a White Paper ABSTRACT on Modernizing Enterprise Systems.


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


Answer:
Generate a White Paper ABSTRACT on Modernizing Enterprise Systems.
The current enterprise systems are largely outdated and not equipped to meet the demands of modern businesses. This white paper discusses the challenges faced by businesses in maintaining their systems and provides a roadmap for modernizing their systems. It also highlights the benefits of adopting a cloud-based approach and cloud-native technologies. The paper also covers the steps involved in the modernization process and the challenges that businesses may face during the process. Finally, the white paper also provides a list of best practices that businesses can follow to successfully modernize their systems.
Question2:
Generate a White Paper MAINCONTENT on Bitcoin Mining.
Answer:
Generate a White Paper MAINCONTENT on Bitcoin Mining.
Bitcoin mining is a complex process that involves the use of specialized hardware to solve complex mathematical equations in order to validate the transactions that are added to 

# Save the Model

In [17]:
if (save_trained_model):
  trainer.save_model(trained_model_id)
  !huggingface-cli login --token $HF_TOKEN
  model.push_to_hub("rai-sandeep/"+trained_model_id, create_pr=1)

In [18]:
def prompt_and_generate():

  doc_type = input("What type of document do you want to generate? (Enter White Paper or Press Release)\n")
  topic = input(f"On what topic do you want to generate the {doc_type}?\n")
  keywords = input(f"Enter any keywords that should be part of the {doc_type} in comma separated format. If none, press Enter.\n")
  content_to_use = input(f"Enter any content that should be used for the {doc_type}. If none, press Enter.\n")

  doc_content=""
  doc_parts=["ABSTRACT","INTRODUCTION","MAINCONTENT","CONCLUSION"]

  for doc_part in doc_parts:
    prompt = f"Generate a {doc_type} {doc_part} on {topic}."
    if keywords.strip() != "":
      prompt += f"\nInclude the following keywords: {keywords}."
    if content_to_use.strip() != "":
      prompt += f"\nUse the following context: {content_to_use}.\n"

    encoding = tokenizer(prompt, return_tensors="pt").to("cuda:0")
    output = model.generate(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask, max_new_tokens=max_tok, do_sample=True, temperature=0.000001, eos_token_id=tokenizer.eos_token_id, top_k = 0)

    temp_content=tokenizer.decode(output[0], skip_special_tokens=True)
    temp_content=temp_content.replace(prompt, "")
    doc_content+=f"\n## {doc_part}  \n{temp_content}  \n"

  print(doc_content)

  f = open(topic.replace(" ", "-")+".md", "w")
  f.write(doc_content)
  f.close()

In [19]:
prompt_and_generate()

What type of document do you want to generate? (Enter White Paper or Press Release)
White Paper
On what topic do you want to generate the White Paper?
Enterprise Test Automation
Enter any keywords that should be part of the White Paper in comma separated format. If none, press Enter.

Enter any content that should be used for the White Paper. If none, press Enter.



Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


##ABSTRACT  

Enterprise test automation is a process of automating the entire test cycle of an application. It involves the creation of test scripts that can be executed without the intervention of a human tester. This approach helps in reducing the time to market of the application, as well as the cost of testing. It also helps in improving the quality of the application by testing it at an early stage of the development cycle. The paper discusses the various components of enterprise test automation, the challenges faced by organizations in implementing it, and the benefits that can be achieved by implementing it. The paper also provides a roadmap for implementing enterprise test automation. The roadmap includes the steps that an organization can follow to successfully implement this approach. The paper also includes a case study of a successful implementation of enterprise test automation for a large financial services company. The case study highlights the challenges faced by the c